In [88]:
# get the data
import pandas as pd
from sklearn.model_selection import train_test_split as ttt

# first we get the data
testPath = "Data/test.csv"
trainPath = "Data/train.csv"
genderSubmissionPath = "Data/gender_submission.csv"

testPDRaw = pd.read_csv(testPath)
trainPDRaw = pd.read_csv(trainPath)
genderSubPDRaw = pd.read_csv(genderSubmissionPath)

# merge the test data too
testDataMerge= pd.merge(genderSubPDRaw,testPDRaw,on = 'PassengerId')


In [89]:
# merge test and train data in order to do the same changes
combine = [trainPDRaw,testPDRaw]
# unuseful data might be the passanger ticket
# remove passanger ticket
for dataset in combine:
    dataset.drop( columns= ['Ticket'], inplace = True)


In [90]:
# considering the names we could reduce it Mr, Mrs, Miss, Col, Master to make it easier
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(trainPDRaw.Title,trainPDRaw.Survived)


Survived,0,1
Title,,
Capt,1,0
Col,1,1
Countess,0,1
Don,1,0
Dr,4,3
Jonkheer,1,0
Lady,0,1
Major,1,1
Master,17,23


In [91]:
# pd.crosstab(trainPDRaw.Title,trainPDRaw.Survived)
# we replace: mlle and Ms by miss, 
for dataset in combine:
    dataset.Title.replace(['Ms', 'Mlle','Mme'], 'Miss', inplace = True)
    # we considered these individuals to be in a lower social ranking so that's why the difference between the 'rare' class
    dataset.Title.replace(['Capt','Col','Rev', 'Dr'],'Other', inplace = True) 
    dataset.Title.replace(['Countess','Don','Dona','Jonkheer','Major','Lady','Sir'], 'Rare', inplace = True)


    dataset.Title.replace( 'Miss','0', inplace = True)
    dataset.Title.replace( 'Mr','1', inplace = True)
    dataset.Title.replace( 'Mrs','2', inplace = True)
    dataset.Title.replace( 'Master','3', inplace = True)
    dataset.Title.replace( 'Other','4', inplace = True)
    dataset.Title.replace( 'Rare','5', inplace = True)

    dataset.Title= dataset.Title.astype(int)

    dataset.Embarked.replace( 'S','0', inplace = True)
    dataset.Embarked.replace( 'Q','1', inplace = True)
    dataset.Embarked.replace( 'C','2', inplace = True)
    

pd.crosstab(trainPDRaw.Title, trainPDRaw.Sex)
# print(testPDRaw.head)

Sex,female,male
Title,,
0,186,0
1,0,517
2,125,0
3,0,40
4,1,15
5,2,5


In [92]:
# see where are we at
# now that we have modified the name into title we drop the name column

for data in combine:
    data.drop(columns= ['Name'], inplace = True)

In [93]:
# we replace male and female by 1 and 0
for data in combine:
    data.Sex.replace('female', '1',inplace=True)
    data.Sex.replace('male','0', inplace =True)
    

In [94]:
# we raw the histogram for the fares
import matplotlib.pyplot as plt

# plt.hist(combine[0].Fare)
# combine[0].Fare.describe()
# we round the columns of fare to see better
for data in combine:
    data.Fare = data.Fare.round(decimals = 0)

In [95]:
# does it make sense to take into account the ticket even though the class is already detailed?
# we createa fare ranges

for dataset in combine:
    dataset.loc[dataset['Fare'] <= 3, 'Fare' ] = 0
    dataset.loc[(dataset['Fare'] <=8) & (dataset['Fare'] >3), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] <=12) & (dataset['Fare'] >8), 'Fare'] = 2
    dataset.loc[(dataset['Fare'] <30) & (dataset['Fare'] >12), 'Fare'] = 3
    dataset.loc[dataset['Fare'] >=30,  'Fare'] = 4

print(combine[0].head)

# farePclass=pd.crosstab(combine[0].Fare,combine[0].Pclass)
# fareSurv = pd.crosstab(combine[0].Fare,combine[0].Survived)


<bound method NDFrame.head of      PassengerId  Survived  Pclass Sex   Age  SibSp  Parch  Fare Cabin  \
0              1         0       3   0  22.0      1      0   1.0   NaN   
1              2         1       1   1  38.0      1      0   4.0   C85   
2              3         1       3   1  26.0      0      0   1.0   NaN   
3              4         1       1   1  35.0      1      0   4.0  C123   
4              5         0       3   0  35.0      0      0   1.0   NaN   
..           ...       ...     ...  ..   ...    ...    ...   ...   ...   
886          887         0       2   0  27.0      0      0   3.0   NaN   
887          888         1       1   1  19.0      0      0   4.0   B42   
888          889         0       3   1   NaN      1      2   3.0   NaN   
889          890         1       1   0  26.0      0      0   4.0  C148   
890          891         0       3   0  32.0      0      0   1.0   NaN   

    Embarked  Title  
0          0      1  
1          2      2  
2          0   

In [96]:
# we drop the cabin and the embarked port
for data in combine:
    data.drop( columns=['Cabin'], inplace = True)


In [97]:
# print(combine[1].Embarked.value_counts())
# print(combine[1].isna().sum())

# we have to deal with the fact that there's a number for age that we should fill, probably with class and fare 
# get the age mean of the different classes and variance 
# do the same with the fare
# for that we use group by, use it on the train sample
ageClasses = combine[0].groupby('Pclass').mean()
ageClassesVar = combine[0].groupby('Pclass').std()
ageFare = combine[0].groupby('Fare').mean()
ageFareVar = combine[0].groupby('Fare').std()
# we get the correlation matrix: the passenger class has a higher correlation with the age, hence it would make sense to model it accordingly

corrMat = combine[0].corr('pearson')

# print(ageClassesVar)
# print(ageClasses)
# print(ageFareVar)
# print(ageFare)
# print(corrMat)

In [98]:
# store the variance and mean of pclass
# meanPclassDF = ageClasses.Age
# varPclassDF = ageClassesVar.Age

# for data in combine:
#     condC1 = (data.Pclass == 1 )& (data.Age.isnull())
#     condC2 = (data.Pclass == 2 )& (data.Age.isnull())
#     condC3 = (data.Pclass == 3 )& (data.Age.isnull())

#     data.Age.replace(condC1, )
# # now we replace nan for the mean and standard deviation

# print(combine[0].loc[(combine[0].Pclass == 1 ) 
# & (combine[0].Age.isnull()) 
# ])


In [99]:
# by doing a combination of pclass and sex we get the age
import numpy
age_guess = numpy.zeros((2,3))
age_guessMed = numpy.zeros((2,3))
for dataset in combine:
    dataset.Pclass = dataset.Pclass.astype(int)
    dataset.Sex = dataset.Sex.astype(int)
    # dataset.Age = dataset.Age.astype(int)

    dataset.Pclass.astype(int,copy = False)
    for i in range(0,2): # sex
        for j in range(1,4): # pclass
            ageGuess = dataset.loc[(dataset['Sex'] == i) & (dataset['Pclass'] == j)]['Age'].dropna()
            
            # print(dataset.Sex.dtype)
            # print(dataset.loc[dataset['Fare'] <= 3, 'Fare' ])
            age_guessMean =ageGuess.mean()
            ageMed = ageGuess.median()
            age_guess[i,j-1] = int(age_guessMean)
            age_guessMed[i,j-1]= int(ageMed)
    
    for i in range(0,2): # sex
        for j in range(0,3): # pclass     
            dataset.Age.loc[(dataset.Age.isnull()) & ( dataset.Sex == i) & (dataset.Pclass == j+1)] = age_guess[i,j]   
    dataset['Age'] = dataset['Age'].astype(int)
            # print( dataset.loc[dataset['Age'].isnull()])

    print(trainPDRaw.head())

   PassengerId  Survived  Pclass  Sex  Age  SibSp  Parch  Fare Embarked  Title
0            1         0       3    0   22      1      0   1.0        0      1
1            2         1       1    1   38      1      0   4.0        2      2
2            3         1       3    1   26      0      0   1.0        0      0
3            4         1       1    1   35      1      0   4.0        0      2
4            5         0       3    0   35      0      0   1.0        0      1
   PassengerId  Survived  Pclass  Sex  Age  SibSp  Parch  Fare Embarked  Title
0            1         0       3    0   22      1      0   1.0        0      1
1            2         1       1    1   38      1      0   4.0        2      2
2            3         1       3    1   26      0      0   1.0        0      0
3            4         1       1    1   35      1      0   4.0        0      2
4            5         0       3    0   35      0      0   1.0        0      1


c:\Users\baldo\Documents\Progra\kaggleTitanic\venv\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
c:\Users\baldo\Documents\Progra\kaggleTitanic\venv\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [100]:
# we create age band
# 13- 19 teenager
# baby 0-4
# child 5-12
# young adult 20-30
# adult until 50
# older than 50

for dataset in combine:
    dataset.Age= dataset.Age.astype(int)
    dataset.loc[dataset['Age'] <= 3, 'Age' ] = 0 #'baby'
    dataset.loc[(dataset['Age'] <= 12) & (dataset.Age >3), 'Age' ] = 1 # child
    dataset.loc[(dataset['Age'] <= 19) & (dataset.Age >12), 'Age' ] = 2  #'teenager'
    dataset.loc[(dataset['Age'] <= 30) & (dataset.Age >19), 'Age' ] = 3#'young'
    dataset.loc[(dataset['Age'] <= 50) & (dataset.Age >30), 'Age' ] =4 #'adult'
    dataset.loc[dataset['Age'] > 50, 'Age' ] = 5#'senior'





In [101]:
print(trainPDRaw.isnull().sum())
# in the train theres two rows that does not have a embarked
# in the test there's one row that does not have the fare
# so we remove both
trainPDRaw.dropna(subset =['Embarked'], inplace= True)


PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       2
Title          0
dtype: int64


In [102]:
# we got some values that are nan so we remove them

# trainPDRaw.dropna(subset = ['Embarked'], inplace= True)
# print(trainPDRaw.isnull().sum())
# 3 we must do the join between the test set and the survived, with the id of the passenger
testDataMergeFinal= pd.merge(genderSubPDRaw,testPDRaw,on = 'PassengerId')
testDataMergeFinal.dropna(subset= ['Fare'], inplace = True)
# print(testDataMergeFinal.isnull().sum())


In [103]:
# models
# prepare the data
from sklearn.model_selection import train_test_split as ttt

yTest = testDataMergeFinal.Survived
xTest = testDataMergeFinal.drop(['Survived','PassengerId'], axis = 1)

xTrain =trainPDRaw.drop(['Survived','PassengerId'],axis = 1 )
yTrain = trainPDRaw.Survived

# split train data
xSplit_Train,xSplit_Test,ySplit_Train,ySplit_Test = ttt(xTrain,yTrain,test_size=0.3,random_state= 0)




In [104]:
# try some models

# lda
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

clfLDA = LDA()

clfLDA.fit(xSplit_Train,numpy.array(ySplit_Train).reshape(numpy.array(ySplit_Train).size))
ySplit_pred = clfLDA.predict(xSplit_Test)
# why functions accuracy does not work

# accuracy = functions.getAccuracy(ySplit_pred,ySplit_Test)
expected_Reshaped = numpy.array(ySplit_Test).reshape(ySplit_Test.size)
test = numpy.array(expected_Reshaped == numpy.array(ySplit_pred)).sum()
accuracy  = test /expected_Reshaped.size *100

print(accuracy)


# definitive test
yDef_pred = clfLDA.predict(xTest)
expected_Reshaped = numpy.array(yTest).reshape(yTest.size)
test = numpy.array(expected_Reshaped == numpy.array(yDef_pred)).sum()
accuracy  = test /expected_Reshaped.size *100
print(accuracy)



77.90262172284645
96.40287769784173
